# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" />

# LiNABiotect
<hr/>
## Instalação do Ambiente

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.9.2"
JULIA_PACKAGES="IJulia Plots Lux Optim OptimizationOptimisers OptimizationOptimJL Optimization DifferentialEquations SciMLSensitivity Flux ComponentArrays Random"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.9.2 on the current Colab Runtime...
2023-07-22 11:49:59 URL:https://julialang-s3.julialang.org/bin/linux/x64/1.9/julia-1.9.2-linux-x86_64.tar.gz [146384758/146384758] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package Plots...
Installing Julia package Lux...
Installing Julia package Optim...
Installing Julia package OptimizationOptimisers...
Installing Julia package Optimization...
Installing Julia package DifferentialEquations...
Installing Julia package SciMLSensitivity...
Installing Julia package Flux...
Installing Julia package ComponentArrays...
Installing Julia package Random...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.9

Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then
jump to the 'Checking the Installation' section.


# Checando se deu tudo certo
A função `versioninfo()` deve imprimir a versão de Julia e outras informações sobre o sistema:

In [1]:
versioninfo()

Julia Version 1.9.2
Commit e4ee485e909 (2023-07-05 09:39 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, znver2)
  Threads: 3 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  JULIA_NUM_THREADS = 2


# Modelo da reação enzimática

## Definition of toy model

<p align="center">
  <img width="480px" src="https://doc-0c-2s-docs.googleusercontent.com/docs/securesc/lmr8anjbmnqqdjm5o3nqommu2igs10a1/h1o5jjnpohrnj88tb3b74bsukek9otfk/1690029675000/06277837039359635046/06277837039359635046/12CRmVFWaW6ZbT8F_9cajfQGlB_WbRiM_?e=download&ax=AGtFMPWNZOUmcabdhLtKoOTeCCcD7VAgs3sE1I8z_L9iwLbiWAmQxUGOHM_ZotiFD3e3vo2yVzKXWuvvWNTnhWESg-2NAQ2Qd8SDR4s1BFdGBAYVZ-M3o_B6BrjljraWr_XAnm9CA6GC7U0Rwg2eascRX-3PNk5gCknn3Pj3Ebf-3EVlWL6_1ZUyYvCAwpGizYqwGpWi4XkdlzLtByFb7dT-oNJ8y5SKNvRJMxAWPbJ1lBHJe8Hqd12LUp3CqqcCmzBA6-mh3Y54gUFrYrAvsfWVPgDVo-tShKCY6wYcP3_gbjuwYLDHam-10RKCVZYgKhnbyIms8qPBpZ-rS1Wn24ghEmiZ3wzj76ugWjwF-Z7A0_X0XKeU434ruC1j1BFdqOOSmRlEbu9uK0B0WMD7Qkelbw2G75Ytz_HugT15MBATBYACGzP_tWvQ2rK4OM6nJczkU-1ZVkJRS1Xt7qWnM3uR5XUDukw3kMwvzlPyo7yAAyocbYwKQiMAjdOLmxon-OzfiWJahSvtg9v4YJ-irb2qL2d6-VEvvPy-DwYCFYGFYwZvyuXaYLeWVk1aObxtbSVdpXrl_6tvYEOoSL4Vb5Y5i-FOOuMchuLctgkaK04QEw1MRQ9AZQAifx99bqjoTSv8LjiqX-9-VXgO1BeNk400YB0Bo8nlI1DtVOMD4oncUS42UffaURmg9rRfYNaBciM6OY02mZRdAo1RoZvP7n_3fftc9a9UyYeLPKR5nPyQ14FhQjk-I_uqbe1Maes5EBOdV4aFxgWjeP21dSIdq_KkjNP6XEsjgELTA6algmWTp5vTD11h8zKcq2BlCAtawA9xkdalkqSPkuFDvFswyQfLs1JWaGMVsCaw89OyUtl03hdUtpur2TPY6zlNFs8ts2rDmDNcwH9EMF7osvOl_lQOuYr0pG9xYarWRsOF&uuid=2659bafd-eeb2-4705-a19c-ff18743a2fd4&authuser=0&nonce=ad4t946518vki&user=06277837039359635046&hash=up1fn19qjjnk5er6l5j86m0b3kgltr1d"/>
</p>

Resulting on the following reactions:

$$
\require{mhchem}
\begin{equation}
    E + S \xrightleftharpoons[k_{r1}]{k_{f1}} ES \xrightarrow{k_{cat1}} E + P\\
\end{equation}
$$

### Simulação

## Plot